In [18]:
%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 1 ms


In [19]:
import pandas as pd
import numpy as np
import glob, shutil
import timeit, os, gc
import subprocess as sp
from tqdm import tqdm
from collections import defaultdict
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor
import json
from IPython.display import HTML
from base64 import b64encode
import cv2



time: 1 ms


In [20]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 4000)
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(font_scale=1.0)

time: 4 ms


In [28]:
num = 1
HOME = "./"
FFMPEG = "C:/Ffmpeg/bin"
FFMPEG_PATH = FFMPEG
DATA_FOLDER = "C:/Users/jamesjun/Desktop/Simone/DF"
#D:\CueLearning\DFData\Df4Extracted
TMP_FOLDER =  "D:/CueLearning/DFData/DfExtracted"
DATA_FOLDER_TRAIN = DATA_FOLDER
VIDEOS_FOLDER_TRAIN = DATA_FOLDER_TRAIN + "/dfdc_train_part_"+str(num)
IMAGES_FOLDER_TRAIN = TMP_FOLDER + "/images"+str(num)
AUDIOS_FOLDER_TRAIN = TMP_FOLDER + "/audios"+str(num)
CROPPED_IMAGES_FOLDER = TMP_FOLDER + "/cropped_images"+str(num)
EXTRACT_META = True # False
EXTRACT_CONTENT = True # False
EXTRACT_FACES = True # False
FRAME_RATE = 3.0 # Frame per second to extract (max is 30.0)

time: 2 ms


In [29]:
def run_command(*popenargs, **kwargs):
    closeNULL = 0
    try:
        from subprocess import DEVNULL
        closeNULL = 0
    except ImportError:
        import os
        DEVNULL = open(os.devnull, 'wb')
        closeNULL = 1

    process = sp.Popen(stdout=sp.PIPE, stderr=DEVNULL, *popenargs, **kwargs)
    output, unused_err = process.communicate()
    retcode = process.poll()

    if closeNULL:
        DEVNULL.close()

    if retcode:
        cmd = kwargs.get("args")
        if cmd is None:
            cmd = popenargs[0]
        error = sp.CalledProcessError(retcode, cmd)
        error.output = output
        raise error
        
    return output

def ffprobe(filename, options = ["-show_error", "-show_format", "-show_streams", "-show_programs", "-show_chapters", "-show_private_data"]):
    ret = {}
    command = [FFMPEG_PATH + "/ffprobe", "-v", "error", *options, "-print_format", "json", filename]
    ret = run_command(command)
    if ret:
        ret = json.loads(ret)
    return ret

# ffmpeg -i input.mov -r 0.25 output_%04d.png
def ffextract_frames(filename, output_folder, rate = 0.25):
    command = [FFMPEG_PATH + "/ffmpeg", "-i", filename, "-r", str(rate), "-y", output_folder + "/output_%04d.png"]
    ret = run_command(command)
    return ret

# ffmpeg -i input-video.mp4 output-audio.mp3
def ffextract_audio(filename, output_path):
    command = [FFMPEG_PATH + "/ffmpeg", "-i", filename, "-vn", "-ac", "1", "-acodec", "copy", "-y", output_path]
    ret = run_command(command)
    return ret

time: 5 ms


In [30]:
# Extract some meta-data
if EXTRACT_META == True:
    results = []
    subfolder = VIDEOS_FOLDER_TRAIN
    filepaths = glob.glob(subfolder + "/*.mp4")
    for filepath in tqdm(filepaths):
        js = ffprobe(filepath)
        if js:
            if len(js.get("streams", [{}, {}]))>1 : 
                results.append(
                    (js.get("format", {}).get("filename")[len(subfolder) + 1:],
                    js.get("format", {}).get("format_long_name"),
                    # Video 
                    js.get("streams", [{}, {}])[0].get("codec_name"),
                    js.get("streams", [{}, {}])[0].get("height"),
                    js.get("streams", [{}, {}])[0].get("width"),
                    js.get("streams", [{}, {}])[0].get("nb_frames"),
                    js.get("streams", [{}, {}])[0].get("bit_rate"),
                    js.get("streams", [{}, {}])[0].get("duration"),
                    js.get("streams", [{}, {}])[0].get("start_time"),
                    js.get("streams", [{}, {}])[0].get("avg_frame_rate"),
                    # Audio
                    js.get("streams", [{}, {}])[1].get("codec_name"),
                    js.get("streams", [{}, {}])[1].get("channels"),
                    js.get("streams", [{}, {}])[1].get("sample_rate"),
                    js.get("streams", [{}, {}])[1].get("nb_frames"),
                    js.get("streams", [{}, {}])[1].get("bit_rate"),
                    js.get("streams", [{}, {}])[1].get("duration"),
                    js.get("streams", [{}, {}])[1].get("start_time")),
                )
            else :
                results.append(
                    (js.get("format", {}).get("filename")[len(subfolder) + 1:],
                    js.get("format", {}).get("format_long_name"),
                    # Video 
                    js.get("streams", [{}, {}])[0].get("codec_name"),
                    js.get("streams", [{}, {}])[0].get("height"),
                    js.get("streams", [{}, {}])[0].get("width"),
                    js.get("streams", [{}, {}])[0].get("nb_frames"),
                    js.get("streams", [{}, {}])[0].get("bit_rate"),
                    js.get("streams", [{}, {}])[0].get("duration"),
                    js.get("streams", [{}, {}])[0].get("start_time"),
                    js.get("streams", [{}, {}])[0].get("avg_frame_rate"),
                    # Audio
                    np.nan,
                    np.nan,
                    np.nan,
                    np.nan,
                    np.nan,
                    np.nan,
                    np.nan),
                )

    meta_pd = pd.DataFrame(results, columns=["filename", "format", "video_codec_name", "video_height", "video_width",
                                            "video_nb_frames", "video_bit_rate", "video_duration", "video_start_time","video_fps",
                                            "audio_codec_name", "audio_channels", "audio_sample_rate", "audio_nb_frames",
                                            "audio_bit_rate", "audio_duration", "audio_start_time"])
    meta_pd["video_fps"] = meta_pd["video_fps"].apply(lambda x: float(x.split("/")[0])/float(x.split("/")[1]) if len(x.split("/")) == 2 else None)
    meta_pd["video_duration"] = meta_pd["video_duration"].astype(np.float32)
    meta_pd["video_bit_rate"] = meta_pd["video_bit_rate"].astype(np.float32)
    meta_pd["video_start_time"] = meta_pd["video_start_time"].astype(np.float32)
    meta_pd["video_nb_frames"] = meta_pd["video_nb_frames"].astype(np.float32)
    meta_pd["video_bit_rate"] = meta_pd["video_bit_rate"].astype(np.float32)
    meta_pd["audio_sample_rate"] = meta_pd["audio_sample_rate"].astype(np.float32)
    meta_pd["audio_nb_frames"] = meta_pd["audio_nb_frames"].astype(np.float32)
    meta_pd["audio_bit_rate"] = meta_pd["audio_bit_rate"].astype(np.float32)
    meta_pd["audio_duration"] = meta_pd["audio_duration"].astype(np.float32)
    meta_pd["audio_start_time"] = meta_pd["audio_start_time"].astype(np.float32)
    meta_pd.to_pickle(HOME + "videos_meta.pkl")
else:
    meta_pd = pd.read_pickle(HOME + "videos_meta.pkl")
meta_pd.head()

0it [00:00, ?it/s]


,filename,format,video_codec_name,video_height,video_width,video_nb_frames,video_bit_rate,video_duration,video_start_time,video_fps,audio_codec_name,audio_channels,audio_sample_rate,audio_nb_frames,audio_bit_rate,audio_duration,audio_start_time


time: 61 ms


In [24]:
train_pd = pd.read_json(VIDEOS_FOLDER_TRAIN + "/metadata.json").T.reset_index().rename(columns={"index": "filename"})
train_pd.head()

,filename,label,split,original
0,jgaejvfomy.mp4,FAKE,train,zqfntxsnna.mp4
1,yaroumkbcf.mp4,FAKE,train,dnocqlgupe.mp4
2,bgxjabrhbg.mp4,FAKE,train,kviesaqfdo.mp4
3,ayneiahymh.mp4,FAKE,train,zqwyggtgfc.mp4
4,uxvuxoabmi.mp4,FAKE,train,jjufjipkit.mp4


time: 867 ms


In [25]:
train_pd = pd.merge(train_pd, meta_pd[["filename", "video_height", "video_width", "video_nb_frames", "video_bit_rate", "audio_nb_frames"]], on="filename", how="left")
train_pd["count"] = train_pd.groupby(["original"])["original"].transform('count')
# train_pd.to_pickle(HOME + "train_meta.pkl")
train_pd.head()

,filename,label,split,original,video_height,video_width,video_nb_frames,video_bit_rate,audio_nb_frames,count
0,jgaejvfomy.mp4,FAKE,train,zqfntxsnna.mp4,1080,1920,300.0,2613943.0,432.0,13.0
1,yaroumkbcf.mp4,FAKE,train,dnocqlgupe.mp4,1080,1920,300.0,6758835.0,432.0,18.0
2,bgxjabrhbg.mp4,FAKE,train,kviesaqfdo.mp4,1920,1080,300.0,6402011.0,432.0,4.0
3,ayneiahymh.mp4,FAKE,train,zqwyggtgfc.mp4,1080,1920,300.0,2411550.0,432.0,18.0
4,uxvuxoabmi.mp4,FAKE,train,jjufjipkit.mp4,1080,1920,300.0,5742318.0,432.0,18.0


time: 27 ms


55552

time: 2 ms


In [16]:
"""videos_folder = VIDEOS_FOLDER_TRAIN
images_folder_path = IMAGES_FOLDER_TRAIN
audios_folder_path = AUDIOS_FOLDER_TRAIN""";

time: 1 ms


In [17]:
AUDIO_FORMAT = "wav" # "wav"
videos_folder = VIDEOS_FOLDER_TRAIN
images_folder_path = IMAGES_FOLDER_TRAIN
audios_folder_path = AUDIOS_FOLDER_TRAIN
if EXTRACT_CONTENT == True:
    # 1h20min for chunk#0 (11GB)
    # Extract some images + audio track
    for idx, row in tqdm(train_pd.iterrows(), total=meta_pd.shape[0]):
        try:
            video_path = videos_folder + "/" + row["filename"]
            images_path = images_folder_path + "/" + row["filename"][:-4]
            audio_path = audios_folder_path + "/" + row["filename"][:-4]
            # Extract images
            if not os.path.exists(images_path): os.makedirs(images_path)
            ret = ffextract_frames(video_path, images_path, rate = FRAME_RATE)
            # Extract audio
            #if not os.path.exists(audio_path): os.makedirs(audio_path)
            #ret = ffextract_audio(video_path, audio_path + "/audio." + AUDIO_FORMAT)
        except:
            print("Cannot extract frames/audio for:" + row["filename"])

100%|████████████████████████████████████| 3134/3134 [1:54:55<00:00,  2.20s/it]

time: 1h 54min 55s
